<a href="https://colab.research.google.com/github/Ayonator77/Random-Matrix/blob/main/RandomMatrix_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.ndimage.filters import gaussian_filter1d
import random
from matplotlib.axes import Axes
from typing import Any, Sequence, Union
from numpy.typing import NDArray
import copy

In [ ]:
#Network layer sizes
size_ = 3*32*32
size2_ = 1000
size3_ = 1000

In [ ]:
#cif_dims = [1000 if i == 0 else size2_ if i < 9 else 10  for i in range(10)] #Model dimensions
no_rel = [False for i in range(10)] #Layers without relu
cif_conv = [400 for i in range(10)]
cif_dims = [256*4*4, 1024, 512, 10]

In [ ]:
class cifar_conv(nn.Module):
  def __init__(self, l = 3, without_rel = no_rel, dims= cif_dims):
    super(cifar_conv, self).__init__()
    self.layers = l
    self.without_rel = without_rel
    self.fc = nn.ModuleList()
    self.dims = dims
    # self.conv1 = nn.Conv2d(3,6,2)
    # self.pool = nn.MaxPool2d(2,2)
    # self.conv2 = nn.Conv2d(6,16,5)
    # self.fc.append(nn.Linear(16*5*5, 1000))
    self.network = nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
      nn.BatchNorm2d(64),

      nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
      nn.BatchNorm2d(128),

      nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
      nn.BatchNorm2d(256),
      nn.Flatten()
    )

    #self.fc.append(nn.Linear(256*4*4, 1024))
    current_dim = dims[0]
    for i in range(1, len(dims)):
      self.fc.append(nn.Linear(current_dim, dims[i], bias=True))
      current_dim = dims[i]

  def forward(self, x):
    # x = self.pool(F.relu(self.conv1(x)))
    # x = self.pool(F.relu(self.conv2(x)))
    # x = torch.flatten(x, 1)
    x = self.network(x)
    x = x.float()
    for j in range(self.layers):
      x = F.relu(self.fc[j](x))

    x = F.log_softmax(x, dim=1)
    return (x)

  def getLayers(self):
    return self.layers

  def getWithout(self):
    return self.withoutrel

  def getDims(self):
    return self.dims

In [ ]:
def train(args, m, device, train_loader, optimizer, epoch):
  m.train()
  i = 1
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    i+=1
    correct = 0
    optimizer.zero_grad()
    (output) = m(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if i % args['log_interval'] == 0:
      print(f'Train Epoch: {epoch} [{ batch_idx *len(data)}/{ len(train_loader.dataset)} ({100. *batch_idx/ len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    return (0)

In [ ]:
def test(args, m, device, test_loader):
  m.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      (output) = m(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader)
  print(f'\ntest set: Average loss: {test_loss:4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.0f}%)\n')
  return (100 * correct / len(test_loader.dataset))

In [ ]:
def main_(k, device, m, lr):
  args = {"batch_size": 30,"test_batch_size": 100, "epochs": 10000,  "lr": 0.01, "momentum": 0.05,  "no_cuda": 'false',  "seed": 2,  "log_interval":1, "save_model": 'false'}
  use_cuda = not args['no_cuda'] and torch.cuda.is_available()
  torch.manual_seed(k)
  kwargs = {'num_workers': 1, 'pin_memory':True} if use_cuda else{}

  #Load training data
  train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('../data',train=True,
      download=True, transform=transforms.Compose([transforms.ToTensor()])), batch_size=args['batch_size'], shuffle=True, **kwargs)

  #Load test data
  test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('../data', train=False,
    transform=transforms.Compose([transforms.ToTensor()])), batch_size=args['test_batch_size'], shuffle=True, **kwargs)

  optimizer = optim.SGD(m.parameters(), lr=args['lr'], momentum=args['momentum'])

  for epoch in range(1, args['epochs']+1):
    train(args, m, device, train_loader, optimizer, epoch)
    if epoch % 100 == 0:
      test(args, m , device, test_loader)

  return (args, m , test_loader, train_loader)


In [ ]:
# args = {"batch_size": 30,"test_batch_size": 100, "epochs": 50,  "lr": 0.01, "momentum": 0.05,  "no_cuda": 'false',  "seed": 2,  "log_interval":1, "save_model": 'false'}
# use_cuda = not args['no_cuda'] and torch.cuda.is_available()
# kwargs = {'num_workers': 1, 'pin_memory':True} if use_cuda else{}
# train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('../data',train=True,
#       download=True, transform=transforms.Compose([transforms.ToTensor()])), batch_size=args['batch_size'], shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('../data', train=False,
#   transform=transforms.Compose([transforms.ToTensor()])), batch_size=args['test_batch_size'], shuffle=True, **kwargs)


# device = torch.device("cuda" if use_cuda else "cpu")
# m  = cifar_conv().to(device)
# optimizer = optim.SGD(m.parameters(), lr=args['lr'], momentum=args['momentum'])
# #test(args, m, device, test_loader)
# for epoch in range(1, args['epochs']):
#   train(args, m, device ,train_loader, optimizer, epoch)

In [ ]:
#computes the probability density of the MP distribution at x, given the scale parameter gamma and variance sigma^2
def _marchenko_pastur(x,gamma,sigma=1.0):
    #support boundary of the mp distribution with limit ratio of gamma, sigma^2 variance
    #really lambda_+ and lambda_- in math notation respectively
    largest_eigenval = np.power(sigma*(1 + np.sqrt(1/gamma)),2)
    smallest_eigenval= np.power(sigma*(1 - np.sqrt(1/gamma)),2)

    #in this case the mp distribution wants to return infinite density.
    #I don't think defaulting to zero introduces any bugs, but this should be evaluated more closely
    if x == 0:
        return 0
    if x<=largest_eigenval and x>=smallest_eigenval:
        mp = (1/(2*np.pi*sigma*sigma*x*(1/gamma)))*np.sqrt((largest_eigenval - x)*(x - smallest_eigenval))
    else:
        mp = 0
    return mp

In [ ]:
#computes graph of mp distribution
def marchenko_pastur(n, p, upper_bound=3, spacing=2000, sigma=1.0):
  x_mp_dist = np.linspace(0, upper_bound, spacing)
  y_mp_dist = [_marchenko_pastur(x, n/p, sigma=sigma) for x in x_mp_dist]
  return x_mp_dist, y_mp_dist

In [ ]:
def transform_to_zero_mean_and_unit_std(X):
  m,n = X.shape
  means = np.mean(X, axis=1).reshape(m, 1)
  mean_adjusted_x = np.subtract(X, means)
  stds = np.std(mean_adjusted_x, axis=1).reshape(m,1)
  mean_adjusted_x = np.divide(mean_adjusted_x, stds)
  return mean_adjusted_x

In [ ]:
def eigenval_histogram(X, bins=100):
  p,n = X.shape
  S = (1/n)*np.dot(X, X.T)
  S[~np.isfinite(S)] = 0
  u,v = np.linalg.eig(S)

  #histogram
  hist_heights, hist_bins = np.histogram(u, bins=bins, density=True)
  x = hist_bins[:-1]
  y = hist_heights
  col_width = hist_bins[-1]/len(x)

  return x, y, col_width, u, v

In [ ]:
def marchenko_pastur_comparison(X, histogram_bins=100, dist_upper_bound=5, dist_spacing=200, transform=True):
  p,n = X.shape
  x_mp, y_mp = marchenko_pastur(n, p, upper_bound=dist_upper_bound, spacing=dist_spacing, sigma=1.0)
  y_mp2 = np.array(y_mp)[~(np.isnan(np.array(y_mp)))]
  largestx = x_mp[len(y_mp2)-1]

  if transform:
    X_ = transform_to_zero_mean_and_unit_std(X)
    x,y,w,u,v = eigenval_histogram(X_, bins=histogram_bins)
  else:
    x,y,w,u,v = eigenval_histogram(X, bins=histogram_bins)

  total = 0
  for i in u:
    if i >= largestx:
      total+=1

  return total, u, v


In [ ]:
#D = s2 x s sized matrix
#returns D, keeping only the p'th largest singular values
def removeP_values(D,s, s2, p):
    assert((s2,s) == D.shape)
    (U,S,V)=np.linalg.svd(D)
    B=np.zeros([s2,s])
    u=np.zeros([s2,1])
    v=np.zeros([1,s])
    for i in range(p):
        u[:,0]=U[:,i]
        v[0,:]=V[i,:]
        s=S[i]
        B+=np.matmul(s*u,v)
    return B

In [ ]:
def diffvalues(Btemp, val, m, w):
  total = 0
  for i in range(len(Btemp)):
    super_threshold_indices = abs(Btemp[i]) < val
    total += sum(super_threshold_indices)
    Btemp[i][super_threshold_indices] = 0

  C = torch.from_numpy(Btemp)
  m.fc[w].weight=nn.Parameter(C.float(), requires_grad=True)
  return Btemp, total

In [ ]:
def split(model, w, k, j):
  dims = model.getDims()

  while True:
    if w >= len(dims)-1:
      return model, True

    A = torch.as_tensor(model.fc[w].weight)
    A = A.cpu()
    A = A.detach().numpy()
    pval, er, er2 = marchenko_pastur_comparison(A)

    if pval > 5 and dims[w] > pval+k+600/j and dims[w+1] > pval+k+600/j:
      p = int(pval+600/int(j))
      print(p)
      break

    else:
      w += 1
      if w == len(dims)- 1:
        return model, True

  print(w)
  size = dims[w]
  size2 = dims[w+1]

  B = removeP_values(A, size, size2, p)
  value = 0.01

  B, accuracy, total = diffvalues(B, value, model, w)
  (U,S,V) = np.linalg.svd(B)

  s = np.zeros([size2, size])
  w1 = np.zeros([size2, size])
  w2 = np.zeros([size2, size])
  B = np.zeros([size2,size])

  for i in range(0, p):
    s[i][i] = S[i]**(1/2)

  w2 = np.matmul(s,V)
  w1 = np.matmul(U,s)

  prevstate = model.state_dict()
  layers = model.getLayers()

  for i in range(layers-1, w, -1):
    prevstate[f'fc.{i+1}.weight'] = prevstate[f'fc.{i}.weight']
    prevstate[f'fc.{i+1}.bias'] = prevstate[f'fc.{i}.bias']

  prevstate[f'fc.{w}.weight'] = torch.from_numpy(w2[:p,:])
  prevstate[f'fc.{w+1}.weight'] = torch.from_numpy(w1[:,:p])
  prevstate[f'fc.{w+1}.bias'] = prevstate[f'fc.{w}.bias']
  prevstate[f'fc.{w}.bias'] = prevstate[f'fc.{w}.bias'][:p]

  withoutrel = model.getWithout()
  withoutrel.insert(w, True)

  del model
  dims.insert(w+1, p)
  print(dims)

  modelnew = cifar_conv(layers+1, withoutrel, dims)
  modelnew.load_state_dict(prevstate)

  return modelnew, False

In [ ]:
if __name__ == '__main__':
  use_cuda = not False and torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  model_conv = cifar_conv().to(device)
  (args, model, test_loader, train_loader) = main_(2, device, model_conv, .01)

Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303699
Train Epoch: 2 [0/50000 (0%)]	Loss: 2.317301
Train Epoch: 3 [0/50000 (0%)]	Loss: 2.293716
Train Epoch: 4 [0/50000 (0%)]	Loss: 2.304899
Train Epoch: 5 [0/50000 (0%)]	Loss: 2.296093
Train Epoch: 6 [0/50000 (0%)]	Loss: 2.265981
Train Epoch: 7 [0/50000 (0%)]	Loss: 2.303421
Train Epoch: 8 [0/50000 (0%)]	Loss: 2.304473
Train Epoch: 9 [0/50000 (0%)]	Loss: 2.268087
Train Epoch: 10 [0/50000 (0%)]	Loss: 2.272270
Train Epoch: 11 [0/50000 (0%)]	Loss: 2.278289
Train Epoch: 12 [0/50000 (0%)]	Loss: 2.291926
Train Epoch: 13 [0/50000 (0%)]	Loss: 2.288714
Train Epoch: 14 [0/50000 (0%)]	Loss: 2.234121
Train Epoch: 15 [0/50000 (0%)]	Loss: 2.291225
Train Epoch: 16 [0/50000 (0%)]	Loss: 2.296680
Train Epoch: 17 [0/50000 (0%)]	Loss: 2.248192
Train Epoch: 18 [0/50000 (0%)]	Loss: 2.252677
Train Epoch: 19 [0/50000 (0%)]	Loss: 2.273483
Train Epoch: 20 [0/50000 (0%)]	Loss: 2.240185
Train Epoch: 21 [0/50000 (0%)]	Loss: 2.272261
Train

KeyboardInterrupt: ignored

In [ ]:
x,y2,y = [], [], []
model2 = copy.deepcopy(model)
x.append(0)
initial = test(args, model, device, test_loader)
y2.append(test(args, model2, device, test_loader))
y.append(initial)

for j in range(1, 10):
  for k in range(4):
    model2, end = split(model2, 0, k, j)

    if end:
      break

  y2.append(test(args, model2, device, test_loader))
  test(args, model2, device, test_loader)

NameError: ignored

In [ ]:
model_conv.state_dict